In [2]:
import itertools as it
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers

2025-03-08 17:56:16.105589: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 17:56:16.174886: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 17:56:16.242851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741467376.310092   31790 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741467376.327292   31790 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 17:56:16.474302: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Possible strategies:
        -- Need to be observed to counter:

- fixed sequence of 'n' moves
        -- previous 'n'+1 moves

- (opponent's / self's) last 'n'[0-10] moves
        -- previous n moves

- (opponent's / self's) most common move in the last 'n'[2-1000] matches
        -- ...

- (opponent's / self's) most common 'm-moves'[2-10] sequence in the last 'n'[2-1000] matches
        -- ...

In [3]:
## Some functions that will be used in the main script ##

#function for counting the number of occurences of a substring in a string
def count_substring(s, sub):
    # Initialize count to zero
    cnt = 0

    # Iterate through the string to check for the substring
    for i in range(len(s) - len(sub) + 1):
        if s[i:i + len(sub)] == sub:  # If substring matches, increment count
            cnt += 1
    return cnt

In [4]:
opponent_hist = "RRRPPPSSS" * 10
player_hist = "RPS" * 30
player_hist

'RPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPS'

In [5]:
length = 10

In [6]:
vocab = sorted(['R', 'P', 'S'])
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return [char2idx[c] for c in text]

def int_to_text(ints):
  return ''.join(idx2char[ints])

In [7]:
#vocab of sequences with 2-5 elements
vocab_seq = {}
for i in range(2, 6):
  vocab_seq[i] = []
  for seq in it.product(vocab, repeat=i):
    vocab_seq[i].append(''.join(seq))

vocab_seq[2]

['PP', 'PR', 'PS', 'RP', 'RR', 'RS', 'SP', 'SR', 'SS']

In [ ]:
#crete a list with the moves of the last 'length' matches
#of the opponent
opp_last_moves = {}
for i in range(length):
    opp_last_moves[i] = []
    for j in range(len(opponent_hist) - length):
        opp_last_moves[i].append(opponent_hist[length + j - i])

opponent_last_moves = [opponent_hist[i:i+length] for i in range(len(opponent_hist) - length)]

#of the player
player_last_moves = [player_hist[i:i+length] for i in range(len(opponent_hist) - length)]

last_moves = opponent_last_moves + player_last_moves
last_moves

['RRRPPPSSSR',
 'RRPPPSSSRR',
 'RPPPSSSRRR',
 'PPPSSSRRRP',
 'PPSSSRRRPP',
 'PSSSRRRPPP',
 'SSSRRRPPPS',
 'SSRRRPPPSS',
 'SRRRPPPSSS',
 'RRRPPPSSSR',
 'RRPPPSSSRR',
 'RPPPSSSRRR',
 'PPPSSSRRRP',
 'PPSSSRRRPP',
 'PSSSRRRPPP',
 'SSSRRRPPPS',
 'SSRRRPPPSS',
 'SRRRPPPSSS',
 'RRRPPPSSSR',
 'RRPPPSSSRR',
 'RPPPSSSRRR',
 'PPPSSSRRRP',
 'PPSSSRRRPP',
 'PSSSRRRPPP',
 'SSSRRRPPPS',
 'SSRRRPPPSS',
 'SRRRPPPSSS',
 'RRRPPPSSSR',
 'RRPPPSSSRR',
 'RPPPSSSRRR',
 'PPPSSSRRRP',
 'PPSSSRRRPP',
 'PSSSRRRPPP',
 'SSSRRRPPPS',
 'SSRRRPPPSS',
 'SRRRPPPSSS',
 'RRRPPPSSSR',
 'RRPPPSSSRR',
 'RPPPSSSRRR',
 'PPPSSSRRRP',
 'PPSSSRRRPP',
 'PSSSRRRPPP',
 'SSSRRRPPPS',
 'SSRRRPPPSS',
 'SRRRPPPSSS',
 'RRRPPPSSSR',
 'RRPPPSSSRR',
 'RPPPSSSRRR',
 'PPPSSSRRRP',
 'PPSSSRRRPP',
 'PSSSRRRPPP',
 'SSSRRRPPPS',
 'SSRRRPPPSS',
 'SRRRPPPSSS',
 'RRRPPPSSSR',
 'RRPPPSSSRR',
 'RPPPSSSRRR',
 'PPPSSSRRRP',
 'PPSSSRRRPP',
 'PSSSRRRPPP',
 'SSSRRRPPPS',
 'SSRRRPPPSS',
 'SRRRPPPSSS',
 'RRRPPPSSSR',
 'RRPPPSSSRR',
 'RPPPSSSRRR',
 'PPPSSSRR

In [28]:
#create a list with the most common move in the last 5 and 10 matches, and in all matches
#of the opponent
op_common = {}
for i in [5, 10]:
  op_common[i] = []
  for j in range(length, len(opponent_hist)):
    op_common[i].append(max(set(opponent_hist[j-i:j]), key=opponent_hist[j-i:j].count))
op_common['all'] = []
for i in range(length, len(opponent_hist)):
  op_common['all'].append(max(set(opponent_hist[:i]), key=opponent_hist[:i].count))

#of the player
pl_common = {}
for i in [5, 10]:
  pl_common[i] = []
  for j in range(length, len(opponent_hist)):
    pl_common[i].append(max(set(player_hist[j-i:j]), key=player_hist[j-i:j].count))
pl_common['all'] = []
for i in range(length, len(opponent_hist)):
  pl_common['all'].append(max(set(player_hist[:i]), key=player_hist[:i].count))

op_common['all']

['R',
 'R',
 'R',
 'R',
 'R',
 'P',
 'P',
 'P',
 'S',
 'R',
 'R',
 'R',
 'R',
 'R',
 'P',
 'P',
 'P',
 'S',
 'R',
 'R',
 'R',
 'R',
 'R',
 'P',
 'P',
 'P',
 'S',
 'R',
 'R',
 'R',
 'R',
 'R',
 'P',
 'P',
 'P',
 'S',
 'R',
 'R',
 'R',
 'R',
 'R',
 'P',
 'P',
 'P',
 'S',
 'R',
 'R',
 'R',
 'R',
 'R',
 'P',
 'P',
 'P',
 'S',
 'R',
 'R',
 'R',
 'R',
 'R',
 'P',
 'P',
 'P',
 'S',
 'R',
 'R',
 'R',
 'R',
 'R',
 'P',
 'P',
 'P',
 'S',
 'R',
 'R',
 'R',
 'R',
 'R',
 'P',
 'P',
 'P']

In [29]:
#create a list with the last sequence of 2-5 moves
#of the opponent
op_last_seq = {}
for i in range(2, 6):
  op_last_seq[i] = [opponent_hist[j-i:j] for j in range(length, len(opponent_hist))]

#of the player
pl_last_seq = {}
for i in range(2, 6):
  pl_last_seq[i] = [player_hist[j-i:j] for j in range(length, len(opponent_hist))]

op_last_seq[2]

['SR',
 'RR',
 'RR',
 'RP',
 'PP',
 'PP',
 'PS',
 'SS',
 'SS',
 'SR',
 'RR',
 'RR',
 'RP',
 'PP',
 'PP',
 'PS',
 'SS',
 'SS',
 'SR',
 'RR',
 'RR',
 'RP',
 'PP',
 'PP',
 'PS',
 'SS',
 'SS',
 'SR',
 'RR',
 'RR',
 'RP',
 'PP',
 'PP',
 'PS',
 'SS',
 'SS',
 'SR',
 'RR',
 'RR',
 'RP',
 'PP',
 'PP',
 'PS',
 'SS',
 'SS',
 'SR',
 'RR',
 'RR',
 'RP',
 'PP',
 'PP',
 'PS',
 'SS',
 'SS',
 'SR',
 'RR',
 'RR',
 'RP',
 'PP',
 'PP',
 'PS',
 'SS',
 'SS',
 'SR',
 'RR',
 'RR',
 'RP',
 'PP',
 'PP',
 'PS',
 'SS',
 'SS',
 'SR',
 'RR',
 'RR',
 'RP',
 'PP',
 'PP',
 'PS',
 'SS']

In [11]:
#create a list with the most common sequence of 2-5 moves in the last 5 and 10 matches
#of the opponent
op_common_seq = {}

for i in range(2, 6):
    op_common_seq[i] = {}
    for j in [5, 10]:
        op_common_seq[i][j] = []
        for k in range(length, len(opponent_hist)):
            count_subs = {}
            for seq in vocab_seq[i]:
                count_subs[seq] = count_substring(opponent_hist[k-j:k], seq)
            op_common_seq[i][j].append(max(count_subs, key=count_subs.get))

#of the player
pl_common_seq = {}

for i in range(2, 6):
    pl_common_seq[i] = {}
    for j in [5, 10]:
        pl_common_seq[i][j] = []
        for k in range(length, len(opponent_hist)):
            count_subs = {}
            for seq in vocab_seq[i]:
                count_subs[seq] = count_substring(player_hist[k-j:k], seq)
            pl_common_seq[i][j].append(max(count_subs, key=count_subs.get))

In [12]:
#create a list with the most common sequence of 2-5 moves in all matches
#of the opponent
op_common_seq_all = {}

for i in range(2, 6):
    op_common_seq_all[i] = []
    for k in range(length, len(opponent_hist)):
        count_subs = {}
        for seq in vocab_seq[i]:
            count_subs[seq] = count_substring(opponent_hist[:k], seq)
        op_common_seq_all[i].append(max(count_subs, key=count_subs.get))

#of the player
pl_common_seq_all = {}

for i in range(2, 6):
    pl_common_seq_all[i] = []
    for k in range(length, len(opponent_hist)):
        count_subs = {}
        for seq in vocab_seq[i]:
            count_subs[seq] = count_substring(player_hist[:k], seq)
        pl_common_seq_all[i].append(max(count_subs, key=count_subs.get))

In [ ]:
features = ['asd'].(['a','b','c'],['d','e'],['f','g','hij'])
features

AttributeError: 'list' object has no attribute 'concatenate'

In [ ]:
next_data = []
data = []

for i in range(len(opponent_hist) - length): #list with next play
    next_data.append(opponent_hist[length + i])

for i in range(len(opponent_hist) - length): #list with previous 'length' plays
    data.append(opponent_hist[i : length + i] + player_hist[i : length + i])

k = 0
print(' ', len(next_data), len(next_data[k]), next_data[k], '\n ',
      len(data), len(data[k]), data[k])

  80 1 R 
  80 20 RRRPPPSSSRRPSRPSRPSR


In [80]:
encoded_data = np.array([text_to_int(d) for d in data])
encoded_next = np.array([text_to_int(d)[0] for d in next_data])
print(encoded_data.shape, encoded_next.shape)

(70, 40) (70,)


In [81]:
features_onehot = np.array([tf.keras.utils.to_categorical(feature, num_classes=len(vocab)) for feature in encoded_data])
labels_onehot = tf.keras.utils.to_categorical(encoded_next, num_classes=len(vocab))
features_onehot.shape, labels_onehot.shape

((70, 40, 3), (70, 3))

In [82]:
model = tf.keras.Sequential([
    layers.LSTM(128, input_shape=(features_onehot.shape[1], features_onehot.shape[2])),
    layers.Dense(127, activation='relu'),
    layers.Dense(3, activation='softmax')
])

model.compile(loss='CategoricalCrossentropy',
              optimizer='adam',
              metrics=['Accuracy'])

history = model.fit(
    features_onehot,
    labels_onehot,
    epochs=100)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - Accuracy: 0.2731 - loss: 1.1033
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - Accuracy: 0.2770 - loss: 1.1034
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.3251 - loss: 1.0985
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.3283 - loss: 1.0974
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.4227 - loss: 1.0979
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - Accuracy: 0.3550 - loss: 1.0965
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.3121 - loss: 1.1001
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - Accuracy: 0.3667 - loss: 1.0962
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.3205 - loss: 1.0972
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.3472 - loss: 1.0947
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - Accuracy: 0.3003 - loss: 1.0979
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - Accuracy: 0.3316 - lo

In [92]:
def predict(opponent, player):
    # input: opponent and player histories
    # output: predicted output and it's probability
    m = min(len(opponent), len(player)) #if the inputs have different lengths, 'm' is the smaller length,
    seq = opponent[-m :] + player[-m :] #only use the last 'm' plays of each player
    seq_encoded = np.array([text_to_int(seq)]) #encode sequence of chars into sequence of ints
    seq_onehot = np.array([tf.keras.utils.to_categorical(se, num_classes=len(vocab)) for se in seq_encoded])
        #encode sequence of ints into sequence of one-hots
    pred = model.predict(seq_onehot) #run the prediction. Return list of probabilities for next char
    idx = np.argmax(pred) #idx of the char with max probability, equal to the integer encoding for the chars
    return int_to_text(idx), round(float(pred[0][idx]), 2) #predicted char, probability

predict('RRRPPPSSS', 'RPSRPSR')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


('R', 0.53)